## **深度学习实践作业 2-a**  
### **通过从卷积神经网络 (CNN) 提取特征进行迁移学习**

#### **目标**
本次实践课程的目标是熟悉一个著名且常用的深度卷积神经网络：VGG16（Simonyan & Zisserman，2015）。接下来，将使用 Chatfield 等人（2014）描述的方法，将此网络应用于 15 Scene 数据集。我们将重点关注如何通过预训练网络提取深层特征，并在经典的分类方案中使用这些特征，通过线性支持向量机 (SVM) 进行分类。

## **Section 1 VGG16 架构**
卷积网络 VGG16（Simonyan & Zisserman，2015，见图1）是一个分类模型，训练于 ImageNet 数据集（Russakovsky 等人，2015）。该数据集包含超过100万张图片，分为1000个类别。其网络架构由五个模块组成，每个模块包含以下部分：
- 两层或三层卷积层，用于保持空间维度；
- 一层池化层，用于将空间维度减半。

然后，在每个后续模块中，特征图的数量加倍。网络的最后部分包含三层全连接层。

在 PyTorch 中可以使用以下命令加载预训练的 VGG16 网络：
```python
import torchvision
vgg16 = torchvision.models.vgg16(pretrained=True)
```

给网络输入的图像需要调整为 224 × 224 像素，并使用均值 µ = [0.485, 0.456, 0.406] 和标准差 σ = [0.229, 0.224, 0.225] 进行标准化（这些是基于 ImageNet 训练集计算得出的）。可以使用提供的字典获取包含 1000 个类别的 ImageNet 类别列表。加载字典和图像可以使用以下代码：

```python
import pickle
from PIL import Image
import numpy as np

nameim = "cat" + ".jpg"
img = Image.open(nameim)
plt.imshow(img)

# 加载 ImageNet 类别
imagenet_classes = pickle.load(open('imagenet_classes.pkl', 'rb'))

# 标准化
img = img.resize((224, 224), Image.BILINEAR)
img = np.array(img, dtype=np.float32) / 255
img = img.transpose((2, 0, 1))

# ImageNet 的均值和标准差
mu = torch.Tensor([0.485, 0.456, 0.406])
sigma = torch.Tensor([0.229, 0.224, 0.225])

# 扩展 mu 和 sigma 以匹配图像大小，并计算标准化后的图像
# 你的代码在这里

# 加载预训练的 VGG
vgg16 = torchvision.models.vgg16(pretrained=True)
vgg16.eval()  # 为什么这么做？

# 在 VGG 上进行前向传播
img = np.expand_dims(img, 0)
x = torch.Tensor(img)
y = ...  # TODO: 计算前向传播
y = y.numpy()  # 转换为 numpy 数组

# 获取预测结果（即 ImageNet 类别标签）
# 你的代码在这里
```

- #### 问题：

1. 由于全连接层占据了模型中绝大部分参数，请根据图1中的尺寸估算VGG16的参数总数。
2. VGG16最后一层的输出大小是多少？它对应的是什么？
3. 将网络应用于几张你选择的图像，并对结果进行评论。
- ImageNet 标准化的作用是什么？
- 为什么要将模型设置为评估模式（eval模式）？
4. bonus：可视化在第一次卷积层后获得的多个激活图。我们如何解读这些激活图？

## **Section 2 使用 VGG16 在 15 Scene 数据集上的迁移学习**
### **2.1 方法**

**总体原则**  
此方法的原则相对简单，包含两个步骤：首先，我们将使用一个预训练的神经网络进行特征提取，然后使用这些特征来训练分类网络。

第一个步骤可以看作是 SIFT + 词袋模型（BoW）方法的一种替代方式。目标是为每张图片生成一个向量表示，然后我们可以使用该向量表示进行多种任务，尤其是图像分类。

特征提取的原理是使用一个训练来解决与我们任务不同的任务的网络来生成这种图像表示。在我们的例子中，VGG16 网络经过训练用于 ImageNet 数据集上的图像分类任务。为了生成图像表示，我们将利用所选预训练网络的中间层输出。

**本实验的原则**  
在本实验中，我们将使用 VGG16 网络，并将每张图片表示为从 `relu7` 层（即分类层之前的 ReLU 层）输出的向量。然后，我们将在 15 Scene 数据集的每个类别上训练一个 SVM 分类器，以解决我们的分类任务。

### 问题
5. 为什么不直接在 15 Scene 数据集上训练 VGG16？
6. 为什么在 ImageNet 上的预训练可以帮助 15 Scene 的分类任务？
7. 你认为特征提取方法有哪些局限性？

### **2.2 使用 VGG16 进行特征提取**

为了在 VGG16 的 `relu7` 层提取特征，请创建一个新类 `VGG16relu7`，将 VGG16 的各层复制至 `relu7` 层（见下方代码）。新网络的输出即为所需的特征。若需从其他层提取特征，可以通过修改 `classifier` 字段（或可能是 `features` 字段）来删除更多的层。

输入网络的图像需要调整为正确的形状（并包含正确的通道数！），并使用上文给出的均值和标准差进行归一化处理。

按照此过程，创建矩阵 `Xtrain` 和 `Xtest`，其中每一行对应一个特征向量。同样，每个特征向量将使用 L2 范数进行归一化。

### 问题
8. 特征提取层的不同会带来什么影响？
9. 15 Scene 数据集的图像是黑白的，而 VGG16 需要 RGB 图像。我们可以如何解决这个问题？

```python
class VGG16relu7(nn.Module):
    def __init__(self):
        super(VGG16relu7, self).__init__()
        # 复制整个卷积部分
        self.features = nn.Sequential(*list(vgg16.features.children()))
        # 保留分类器的一部分：-2 表示在 relu7 层停止
        self.classifier = nn.Sequential(*list(vgg16.classifier.children())[:-2])

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x
```

### **2.3 训练 SVM 分类器**

使用提取的特征 (Xtrain, Xtest) 和相应的标签 (ytrain, ytest)，我们将训练一个多类分类器（由多个一对多的二分类线性 SVM 分类器组成）。这项工作之前已经完成；你将使用 Scikit Learn 中的 `sklearn.svm.LinearSVC`。

```python
from sklearn.svm import LinearSVC
svm = LinearSVC(C=1.0)
```

在训练集上使用 `C = 1` 来训练一个多类 SVM 分类器，并使用之前提取的深度特征在测试集上评估其分类准确率。为此，将使用 `LinearSVC` 中的 `fit` 和 `score` 函数来训练 SVM 并计算其性能。

### 问题

10. 与训练一个独立的分类器相比，是否可以直接使用神经网络？请解释。

### **2.4 深入探讨**

一旦这个分类方案工作正常，您可以研究两个（或更多）选择对性能或训练时间的影响。以下是一些可能的例子，但您也可以自由寻找其他的选择！

- 改变特征提取的层次。该层的深度有多重要？特征表示的大小是多少，这种变化会产生什么影响？
- 尝试其他可用的预训练网络。这些网络之间有什么区别？
- 调整参数 C 以提高性能。
- 替代训练 SVM，替换 VGG16 的最后一层为一个新的全连接层，并继续在 15 Scene 上训练网络（可以选择是否将梯度传播到网络的其他部分）。
- 研究在分类前进行降维的方法及其对性能和执行时间的影响。

### 问题
11. 对于您测试的每一个改进，解释您的推理并评论获得的结果。